In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# activate R magic
%load_ext rpy2.ipython

# Classes for latent var ml model

In [3]:
%%R
adjusted.predictors = function( x.data, z.data , x.gid, z.gid) {
  x.data = as.data.frame(x.data)
  z.data = as.data.frame(z.data)
  # sort individual-level data based on group ID
  x = x.data[order(x.gid),]
  # sort group-level data
  z = z.data[order(z.gid),]
  # number of individuals per group
  ng = as.numeric(unlist(tapply(rep(1,dim(x)[1]),x.gid,sum)))
  #1a: Parameters pertaining to distribution of Z
  mu_z = apply( z, 2, mean ) # 1-by-q
  sigma_zz = var( z ) # q-by-q
  #1b: matrix of observed means
  mu_x = apply( x, 2, mean ) # 1-by-p
  mu_xg = apply( x, 2, function(x) {tapply(x, x.gid, mean)} ) # G by p, mean by group, the unadjusted mean
  gid = x.gid
  x.data2 = cbind(x, gid)
  gid = z.gid
  mu_xg2 = cbind(mu_xg, gid)
  x.data.means = merge( x.data2, mu_xg2, by="gid" ) # merge group means with individual level data
  #1c: Covariance matrix of Z and X
  sigma_xiz = cov( mu_xg, z ) # p by q, between-group covariance
  sigma_zxi = t( sigma_xiz ) # q by p, transpose
  #1d: Between- and within-group covariance matrices
  SSA = sum(ng)*apply(mu_xg,1,function(x){x-mu_x})%*%t(apply(mu_xg,1,function(x){x-mu_x})) # p by p
  MSA = SSA/(dim(x)[1]-dim(z)[1]) # p by p
  SSE = matrix( 0, nrow=dim(x)[2], ncol=dim(x)[2]) # p by p
  for (k in 1:(dim(x.data.means)[1])) {
    increment = (dim(x.data.means)[2]-1)/2 # should be the same as the unique individual-level variables
    SSE_i = as.numeric( unlist( x.data.means[k,2:(dim(x.data.means)[2]-increment)] - x.data.means[k,(dim(x.data.means)[2]-increment+1):dim(x.data.means)[2]] ) )%*%t(as.numeric( unlist( x.data.means[k,2:(dim(x.data.means)[2]-increment)] - x.data.means[k,(dim(x.data.means)[2]-increment+1):dim(x.data.means)[2]] ) ))
    SSE = SSE + SSE_i
  }
  MSE = SSE/( dim(z)[1] - 1 )
  sigma_vv = MSE # p by p
  sigma_xixi = ((dim(x)[1]*(dim(z)[1] - 1))/(dim(x)[1]**2-sum(ng**2)))*(MSA-MSE) # p by p
  #2: Adjusted group means
  adjusted.means = matrix( NA, nrow=dim(z)[1], ncol=dim(x)[2] ) # G by p
  for (g in 1:dim(z)[1]) {
    if (dim(z)[2]>0) {
      W_g1 = solve(sigma_xixi + sigma_vv/as.numeric(ng)[g] + sigma_xiz%*%solve(sigma_zz)%*%sigma_zxi)%*%(sigma_xixi + sigma_xiz%*%solve(sigma_zz)%*%sigma_zxi)
      W_g2 = solve(sigma_zz)%*%sigma_zxi%*%(diag(dim(x)[2])-W_g1)
      if (dim(as.matrix(z[g,]-mu_z,nrow=1))[1] == 1){
        addx = as.matrix(z[g,]-mu_z,nrow=1)
      } else {
        addx = t(as.matrix(z[g,]-mu_z,nrow=1))
      }
      adjusted.means[g,] = t(mu_x)%*%(diag(dim(x)[2])-W_g1)+t(mu_xg[g,])%*%W_g1+addx%*%W_g2
    } else {
      W_g1 = solve(sigma_xixi + sigma_vv/as.numeric(ng)[g])%*%(sigma_xixi)
      adjusted.means[g,] = t(mu_x)%*%(diag(dim(x)[2])-W_g1)+t(mu_xg[g,])%*%W_g1
    }
  }
  adjusted.group.means = as.data.frame( cbind( adjusted.means, z, z.gid) )
  names(adjusted.group.means) = c( paste( "BLUP.", names(x), sep="" ), names(z), "gid")

  # assign boolean variable to unequal.groups which returns TRUE when groups sizes are different, FALSE otherwise
  unequal.groups = isTRUE(min(ng) != max(ng))

  group.size = as.data.frame(cbind(z.gid, ng))
  report =  list("unequal.groups" = unequal.groups, "adjusted.group.means" = adjusted.group.means, "group.size" = group.size)
  return(report)
}






In [4]:
%%R
micromacro.lm = function(model, adjusted.predictors, y, unequal.groups=NULL) {
  data = cbind(adjusted.predictors,y)
  uncorrected.output = lm( model, data)
  model.terms = attr(terms.formula(model), "term.labels")
  model.variables = model.terms[!grepl(":",model.terms)]
  interaction.terms = model.terms[grepl(":",model.terms)]
  if (length(interaction.terms)>0) {
    interactions = matrix( NA, nrow=dim(data)[1], ncol=length(interaction.terms) )
    for ( i in 1:length(interaction.terms) ) {
      interactions[,i] = apply( data[strsplit(interaction.terms,":")[[i]]], 1, prod )
    }
    u = cbind(rep(1,dim(data)[1]),as.matrix(cbind(data[model.variables], interactions)))
  } else {u = cbind(rep(1,dim(data)[1]),as.matrix(cbind(data[model.variables])))}
  e = uncorrected.output$residuals
  p = solve(t(u) %*% u)
  h = diag(u %*% p %*% t(u))
  d = e^2/(1-h)
  v = p %*% t(u) %*% diag(d) %*% u %*% p
  se = sqrt(diag(v))
  estim = summary(uncorrected.output)$coefficients[,1:2]
  t.stats = estim[,1]/se
  p.values = pt(-abs(t.stats),rep(uncorrected.output$df.residual,length(model.terms)+1))*2 # two-sided
  effect.sizes = sqrt( t.stats**2/(t.stats**2+uncorrected.output$df.residual) )
  estim1 = cbind(estim,se,rep(uncorrected.output$df.residual,length(model.terms)+1),t.stats,p.values,effect.sizes)
  dimnames(estim1)[[2]] = c("Estimate","Uncorrected S.E.","Corrected S.E.","df","t","Pr(>|t|)","r")
  # print( estim )
  if (is.null(unequal.groups)){ # default (null) is same group size
    t.stats2 = estim[,1]/estim[,2]
    p.values2 = pt(-abs(t.stats2),rep(uncorrected.output$df.residual,length(model.terms)+1))*2
    effect.sizes2 = sqrt( t.stats2**2/(t.stats2**2+uncorrected.output$df.residual) )
    estim2 = cbind(estim,rep(uncorrected.output$df.residual,length(model.terms)+1),t.stats2,p.values2,effect.sizes2)
    dimnames(estim2)[[2]] = c("Estimate","S.E.","df","t","Pr(>|t|)","r")
    report = list("statistics" = estim2, "rsquared" = summary(uncorrected.output)$r.squared, "rsquared.adjusted" = summary(uncorrected.output)$adj.r.squared, "residuals" = uncorrected.output$residuals, "fitted.values" = uncorrected.output$fitted.values, "fstatistic" = summary(uncorrected.output)$fstatistic, "model.formula" = model)
  } else if (isTRUE(unequal.groups)) { # true when group sizes are unequal
    report = list("statistics" = estim1, "rsquared" = summary(uncorrected.output)$r.squared, "rsquared.adjusted" = summary(uncorrected.output)$adj.r.squared, "residuals" = uncorrected.output$residuals, "fitted.values" = uncorrected.output$fitted.values, "fstatistic" = summary(uncorrected.output)$fstatistic, "model.formula" = model)
  } else { # true when group sizes are the same
    t.stats2 = estim[,1]/estim[,2]
    p.values2 = pt(-abs(t.stats2),rep(uncorrected.output$df.residual,length(model.terms)+1))*2
    effect.sizes2 = sqrt( t.stats2**2/(t.stats2**2+uncorrected.output$df.residual) )
    estim2 = cbind(estim,rep(uncorrected.output$df.residual,length(model.terms)+1),t.stats2,p.values2,effect.sizes2)
    dimnames(estim2)[[2]] = c("Estimate","S.E.","df","t","Pr(>|t|)","r")
    report = list("statistics" = estim2, "rsquared" = summary(uncorrected.output)$r.squared, "rsquared.adjusted" = summary(uncorrected.output)$adj.r.squared, "residuals" = uncorrected.output$residuals, "fitted.values" = uncorrected.output$fitted.values, "fstatistic" = summary(uncorrected.output)$fstatistic, "model.formula" = model)
  }
  return(report)
}


In [5]:
%%R

micromacro.summary = function(model.output) {
  cat(noquote("Call:"))
  cat("\n")
  cat(noquote(sprintf("micromacro.lm( %s %s %s, ...)", gettext(model.output$model.formula)[2], gettext(model.output$model.formula)[1], gettext(model.output$model.formula)[3])))
  cat("\n")
  cat(noquote("    "))
  cat("\n")
  # summarize the residuals
  cat(noquote("Residuals:"))
  cat("\n")
  residual.summary = cbind(min(model.output$residuals), quantile(model.output$residuals, 0.25), quantile(model.output$residuals, 0.5), quantile(model.output$residuals, 0.75), max(model.output$residuals))
  dimnames(residual.summary)[[1]] = ""
  dimnames(residual.summary)[[2]] = c("Min","1Q","Median","3Q","Max")
  print(noquote(residual.summary))
  cat("\n")
  cat(noquote("    "))
  cat("\n")
  # summarize the coefficients
  cat(noquote("Coefficients:"))
  cat("\n")
  print(noquote(model.output$statistics))
  cat("\n")
  cat(noquote("---"))
  cat("\n")
  cat(noquote(sprintf("Residual standard error: %s on %s degrees of freedom", round(sd(model.output$residuals), digits = 5), length(model.output$residuals)-dim(model.output$statistics)[1])))
  cat("\n")
  cat(noquote(sprintf("Multiple R-squared: %s, Adjusted R-squared: %s", round(model.output$rsquared, digits = 10), round(model.output$rsquared.adjusted, digits = 10))))
  cat("\n")
  cat(noquote(sprintf("F-statistic: %s on %s and %s DF, p-value: %s", round(model.output$fstatistic[1], digits = 5), model.output$fstatistic[2], model.output$fstatistic[3], round(pf(model.output$fstatistic[1], model.output$fstatistic[2], model.output$fstatistic[3], lower.tail=FALSE), digits = 5))))
}


# US vs asia

In [6]:
%%R

library("dplyr")
load("/content/drive/My Drive/Dissertation/airbnb/signal x hofstede/lvmlm/us vs asia/201804 onward/xdata.RData")
load("/content/drive/My Drive/Dissertation/airbnb/signal x hofstede/lvmlm/us vs asia/201804 onward/zdata.RData")
x.gid <- x[['gid']]
x.data <- select(x, usvasia, host_is_superhost)
#x.data$nothing <- sample(100, size = nrow(x.data), replace = TRUE)
z.gid <- z[['gid']]

data <- z
epsilon <- 0.0000001
percent_perf <- (30-data$availability_30)/30
z$perf <- ifelse(percent_perf == 0, percent_perf+epsilon, percent_perf)
z$perf <- ifelse(percent_perf == 1, percent_perf-epsilon, z$perf)
z$logit_perf <- log(z$perf/(1-z$perf))

#add seasonality
z$jan <- ifelse(substr(z$yyyymm, 5, 6)=="01", 1, 0)
z$feb <- ifelse(substr(z$yyyymm, 5, 6)=="02", 1, 0)
z$mar <- ifelse(substr(z$yyyymm, 5, 6)=="03", 1, 0)
z$apr <- ifelse(substr(z$yyyymm, 5, 6)=="04", 1, 0)
z$may <- ifelse(substr(z$yyyymm, 5, 6)=="05", 1, 0)
z$jun <- ifelse(substr(z$yyyymm, 5, 6)=="06", 1, 0)
z$jul <- ifelse(substr(z$yyyymm, 5, 6)=="07", 1, 0)
z$aug <- ifelse(substr(z$yyyymm, 5, 6)=="08", 1, 0)
z$sep <- ifelse(substr(z$yyyymm, 5, 6)=="09", 1, 0)
z$oct <- ifelse(substr(z$yyyymm, 5, 6)=="10", 1, 0)
z$nov <- ifelse(substr(z$yyyymm, 5, 6)=="11", 1, 0)

y <- z[['logit_perf']]
z.data <- select(z, gid, host_is_superhost, price, number_of_reviews, review_scores_rating, jan, feb, mar, apr, may, jun, jul, aug, sep, oct, nov)
colnames(z.data)[1] <- "group.id"
rm(x, z, data, epsilon, percent_perf)

R[write to console]: 
Attaching package: ‘dplyr’


R[write to console]: The following objects are masked from ‘package:stats’:

    filter, lag


R[write to console]: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [7]:
%%R
results = adjusted.predictors(x.data, z.data, x.gid, z.gid)

R[write to console]: Error in solve.default(sigma_zz) : 
  Lapack routine dgesv: system is exactly singular: U[6,6] = 0




Error in solve.default(sigma_zz) : 
  Lapack routine dgesv: system is exactly singular: U[6,6] = 0


RInterpreterError: ignored

In [10]:
%%R
model.formula = as.formula(y ~ host_is_superhost*BLUP.usvasia
                           + price + review_scores_rating + number_of_reviews
                          )
model.output = micromacro.lm(model.formula, results$adjusted.group.means, y, results$unequal.groups)
micromacro.summary(model.output)

Call:
micromacro.lm( y ~ host_is_superhost * BLUP.usvasia + price + review_scores_rating + number_of_reviews, ...)
    
Residuals:
       Min        1Q     Median       3Q      Max
 -19.02469 -1.363415 -0.3309223 0.639411 17.19515

    
Coefficients:
                                   Estimate Uncorrected S.E. Corrected S.E.
(Intercept)                    -2.838793669     0.5451529112   0.5490479374
host_is_superhost               0.161669173     0.1497281205   0.1530797478
BLUP.usvasia                    0.120848642     0.1134935756   0.1110227799
price                          -0.002072563     0.0003214283   0.0002781555
review_scores_rating            0.040346449     0.0058536676   0.0059352174
number_of_reviews               0.003643062     0.0002785902   0.0002912077
host_is_superhost:BLUP.usvasia -0.037664782     0.1632316041   0.1658866142
                                  df          t     Pr(>|t|)           r
(Intercept)                    25327 -5.1703931 2.353770e-07 0.03247

In [ ]:
%%R
saveRDS(model.output, "/content/drive/My Drive/m1.rds")

In [8]:
%%R
#model.formula = as.formula(y ~ host_is_superhost*number_of_reviews + host_is_superhost:number_of_reviews:BLUP.usvasia 
model.formula = as.formula(y ~ host_is_superhost*BLUP.usvasia 
                           #+ price + review_scores_rating + number_of_reviews
                           #+ jan + feb + mar + apr + may + jun + jul + aug + sep + oct + nov
                           )
model.output = micromacro.lm(model.formula, results$adjusted.group.means, y, results$unequal.groups)
micromacro.summary(model.output)

R[write to console]: Error in cbind(adjusted.predictors, y) : object 'results' not found




Error in cbind(adjusted.predictors, y) : object 'results' not found


RInterpreterError: ignored

In [ ]:
%%R
saveRDS(model.output, "/content/drive/My Drive/m1-1.rds")

# hofstede dim

In [19]:
%%R

library("dplyr")
#load("/content/drive/My Drive/Dissertation/airbnb/signal x hofstede/lvmlm/201810 onward/xdata.RData")
#load("/content/drive/My Drive/Dissertation/airbnb/signal x hofstede/lvmlm/201810 onward/zdata.RData")
load("/content/drive/My Drive/Dissertation/airbnb/signal x hofstede/lvmlm/us vs asia/201804 onward/xdata.RData")
load("/content/drive/My Drive/Dissertation/airbnb/signal x hofstede/lvmlm/us vs asia/201804 onward/zdata.RData")
x.gid <- x[['gid']]
#x.data <- select(x, power_distance, individualism, masculinity, uncertainty_avoidance, LT_orientation, indulgence)
x.data <- select(x, globe1_dst, globe2_dst)
#x.data$nothing <- sample(100, size = nrow(x.data), replace = TRUE)
z.gid <- z[['gid']]

data <- z
epsilon <- 0.0000001
percent_perf <- (30-data$availability_30)/30
z$perf <- ifelse(percent_perf == 0, percent_perf+epsilon, percent_perf)
z$perf <- ifelse(percent_perf == 1, percent_perf-epsilon, z$perf)
z$logit_perf <- log(z$perf/(1-z$perf))

#add seasonality
z$jan <- ifelse(substr(z$yyyymm, 5, 6)=="01", 1, 0)
z$feb <- ifelse(substr(z$yyyymm, 5, 6)=="02", 1, 0)
z$mar <- ifelse(substr(z$yyyymm, 5, 6)=="03", 1, 0)
z$apr <- ifelse(substr(z$yyyymm, 5, 6)=="04", 1, 0)
z$may <- ifelse(substr(z$yyyymm, 5, 6)=="05", 1, 0)
z$jun <- ifelse(substr(z$yyyymm, 5, 6)=="06", 1, 0)
z$jul <- ifelse(substr(z$yyyymm, 5, 6)=="07", 1, 0)
z$aug <- ifelse(substr(z$yyyymm, 5, 6)=="08", 1, 0)
z$sep <- ifelse(substr(z$yyyymm, 5, 6)=="09", 1, 0)
z$oct <- ifelse(substr(z$yyyymm, 5, 6)=="10", 1, 0)
z$nov <- ifelse(substr(z$yyyymm, 5, 6)=="11", 1, 0)

y <- z[['logit_perf']]
z.data <- select(z, gid, host_is_superhost, price, number_of_reviews, review_scores_rating, jan, feb, mar, apr, may, jun, jul, aug, sep, oct, nov)
colnames(z.data)[1] <- "group.id"
rm(x, z, data, epsilon, percent_perf)

In [20]:
%%R
results = adjusted.predictors(x.data, z.data, x.gid, z.gid)

R[write to console]: Error in cov(mu_xg, z) : incompatible dimensions

R[write to console]: In addition: 
R[write to console]: Warning message:

R[write to console]: In cbind(mu_xg, gid) :
R[write to console]: 
 
R[write to console]:  number of rows of result is not a multiple of vector length (arg 2)




Error in cov(mu_xg, z) : incompatible dimensions


RInterpreterError: ignored

In [29]:
%%R
model.formula = as.formula(y ~ host_is_superhost*(BLUP.uncertainty_avoidance + BLUP.power_distance) + BLUP.individualism + BLUP.masculinity + BLUP.LT_orientation + BLUP.indulgence
                           + price + review_scores_rating + number_of_reviews
                          )
model.output = micromacro.lm(model.formula, results$adjusted.group.means, y, results$unequal.groups)
micromacro.summary(model.output)

Call:
micromacro.lm( y ~ host_is_superhost * (BLUP.power_distance) + BLUP.individualism + BLUP.masculinity + BLUP.LT_orientation + BLUP.indulgence, ...)
    
Residuals:
       Min        1Q     Median       3Q      Max
 -17.05568 -1.148504 0.01193346 1.042668 16.11124

    
Coefficients:
                                           Estimate Uncorrected S.E.
(Intercept)                            0.0131000321      0.437092443
host_is_superhost                      0.2906686760      0.187458455
BLUP.power_distance                   -0.0032146550      0.003300051
BLUP.individualism                     0.0014773861      0.002068222
BLUP.masculinity                       0.0006614901      0.002563596
BLUP.LT_orientation                    0.0025686146      0.002139205
BLUP.indulgence                        0.0031894088      0.002914326
host_is_superhost:BLUP.power_distance  0.0046745619      0.003721881
                                      Corrected S.E.    df          t  Pr(>|t|)
(Intercept

In [9]:
%%R
saveRDS(model.output, "/content/drive/My Drive/m1.rds")

In [16]:
%%R
#BLUP.uncertainty_avoidance + BLUP.power_distance + BLUP.individualism + BLUP.masculinity + BLUP.LT_orientation + BLUP.indulgence
model.formula = as.formula(y ~ host_is_superhost*BLUP.globe1_dst 
                           + price + review_scores_rating + number_of_reviews
                           + jan + feb + mar + apr + may + jun + jul + aug + sep + oct + nov
                           )
model.output = micromacro.lm(model.formula, results$adjusted.group.means, y, results$unequal.groups)
micromacro.summary(model.output)

Call:
micromacro.lm( y ~ host_is_superhost * BLUP.globe1_dst + price + review_scores_rating + number_of_reviews + jan + feb + mar + apr + may + jun + jul + aug + sep + oct + nov, ...)
    
Residuals:
       Min        1Q     Median        3Q      Max
 -18.70531 -1.300587 -0.2277505 0.9201892 20.26906

    
Coefficients:
                                       Estimate Uncorrected S.E. Corrected S.E.
(Intercept)                       -3.9557391933     0.4042016825   0.3948145440
host_is_superhost                  0.1189831292     0.1140324966   0.1208717964
BLUP.globe1_dst                   -0.0519732394     0.0431458582   0.0475285195
price                             -0.0005216345     0.0001755434   0.0005066209
review_scores_rating               0.0378649214     0.0042122578   0.0042252125
number_of_reviews                  0.0050260044     0.0002430110   0.0002449276
jan                               -3.0035477453     0.1304083659   0.1476501471
feb                               -0.0

In [ ]:
%%R
saveRDS(model.output, "/content/drive/My Drive/m1-1.rds")

# globe distance

In [33]:
%%R

library("dplyr")
load("/content/drive/My Drive/Dissertation/airbnb/signal x hofstede/lvmlm/us vs asia/201804 onward/xdata.RData")
load("/content/drive/My Drive/Dissertation/airbnb/signal x hofstede/lvmlm/us vs asia/201804 onward/zdata.RData")
x.gid <- x[['gid']]
x.data <- select(x, globe1_dst, globe2_dst)
#x.data$nothing <- sample(100, size = nrow(x.data), replace = TRUE)
z.gid <- z[['gid']]

data <- z
epsilon <- 0.0000001
percent_perf <- (30-data$availability_30)/30
z$perf <- ifelse(percent_perf == 0, percent_perf+epsilon, percent_perf)
z$perf <- ifelse(percent_perf == 1, percent_perf-epsilon, z$perf)
z$logit_perf <- log(z$perf/(1-z$perf))

y <- z[['logit_perf']]
z.data <- select(z, gid, host_is_superhost, host_listings_count, number_of_reviews, price, bathrooms, bedrooms, review_scores_location)
colnames(z.data)[1] <- "group.id"
rm(x, z, data, epsilon, percent_perf)

In [34]:
%%R
results = adjusted.predictors(x.data, z.data, x.gid, z.gid)

In [41]:
%%R
model.formula = as.formula(y ~ host_is_superhost*BLUP.globe1_dst 
                           + host_listings_count + number_of_reviews + price + bathrooms + bedrooms + review_scores_location
                          )
model.output = micromacro.lm(model.formula, results$adjusted.group.means, y, results$unequal.groups)
micromacro.summary(model.output)

Call:
micromacro.lm( y ~ host_is_superhost * BLUP.globe1_dst + host_listings_count + number_of_reviews + price + bathrooms + bedrooms + review_scores_location, ...)
    
Residuals:
      Min        1Q    Median         3Q      Max
 -19.7799 -1.943961 -1.063328 -0.1091585 15.62092

    
Coefficients:
                                      Estimate Uncorrected S.E. Corrected S.E.
(Intercept)                       -3.275104331     1.3054318834   1.4724575857
host_is_superhost                  1.691376554     0.7144078796   0.8274200297
BLUP.globe1_dst                   -0.539765193     0.2361852147   0.2756648821
host_listings_count               -0.005181815     0.0026435656   0.0020827751
number_of_reviews                  0.003108025     0.0007339919   0.0007286982
price                             -0.001973108     0.0007569134   0.0008704797
bathrooms                         -0.050231040     0.0440412464   0.1047479528
bedrooms                          -0.102054686     0.0825084149   0

In [36]:
%%R
saveRDS(model.output, "/content/drive/My Drive/m1.rds")